## Importing Data

The data sets we currently have are split into two categories training data and testing data. Right now, each training data set has 700k rows and each testing data set has 300k rows. That means when we are comparing two data sets there are $700000^2$ many pairs to check, which is huge. There are some duplicates in these data sets, i.e. rows with same NPI's. That's why we will drop the duplicates so that there are less pairs to compare.

In [1]:
import numpy as np 
import pandas as pd

dataA = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataAtrain.csv")
dataB = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataBtrain.csv")
dataC = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataCtrain.csv")

dataA_test = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataAtest.csv")
dataB_test = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataBtest.csv")
dataC_test = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataCtest.csv",encoding="latin1")


/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/471968081.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataA = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataAtrain.csv")
/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/471968081.py:5: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataB = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataBtrain.csv")
/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/471968081.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataA_test = pd.read_csv("/Users/ovunc/Desktop/Civic_Task/dataAtest.csv")
/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/471968081.py:9: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataB_test = pd.read_csv("/Users/ovunc/Desktop

In [2]:
dataA_prov = dataA.drop_duplicates(subset=["NPI"])
dataB_prov = dataB.drop_duplicates(subset=["NPI"])
dataC_prov = dataC.drop_duplicates(subset=["NPI"])

In [3]:
len(dataA_prov)

437579

In [4]:
len(dataB_prov)

300446

In [5]:
len(dataC_prov)

588159

There are still trillions of pairs to compare. So we will apply blocking methods to reduce the number of pairs.

## Blocking Methods

From the exploratory data analysis part, we have seen that three columns that doesn't have any missing values are first/last names and state. So what I first wanted to implement is using blocking methods with those three columns. Then I decided to first work with exact blocking. However, when I tried using those three columns, my kernel stopped working because of the abundance of matching. That's why for exact blocking I decided to use ZIP codes with the other two columns. 

Moreover, while deciding on which blocking method to use we will only focus on the data sets A and B. We will first start by exact blocking using single column: ZIP. Then I will include the initials of first and the last names (The reason for only using initials is because we want to consider typos or different spellings). After each method we will analyze how succesfull that method is.

In [6]:
# Exact Blocking
# Blocking by using only ZIP code

def make_zip5(zip_series):
    return (
        zip_series
        .astype(str)
        .str.extract(r"(\d{5})")[0]
    )

for df in [dataA_prov, dataB_prov]:
    df["zip5"] = make_zip5(df["ZIP"])

block_sizes = (
    dataA_prov.groupby(["zip5"])
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_prov.groupby(["zip5"])
        .size()
        .reset_index(name="dataB_count"),
        on=["zip5"],
        how="inner"
    )
)

block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

block_sizes.sort_values("product", ascending=False).head(10)


/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/3479284249.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["zip5"] = make_zip5(df["ZIP"])
/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/3479284249.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["zip5"] = make_zip5(df["ZIP"])


,zip5,dataA_count,dataB_count,product
9911,77030,1769,1643,2906467
6159,44195,1102,574,632548
10162,78229,754,654,493116
8133,60611,882,519,457758
8447,63110,916,473,433268
1183,10016,757,535,404995
7736,55905,1367,290,396430
2443,19104,846,454,384084
8134,60612,741,493,365313
9772,76104,697,509,354773


In [7]:
# Ground truth matches using NPI
true_matches = (
    dataA_prov[["NPI"]]
    .reset_index()
    .merge(
        dataB_prov[["NPI"]].reset_index(),
        on="NPI",
        how="inner",
        suffixes=("_A", "_B")
    )
)

# Set of true match index pairs
true_match_pairs = set(
    zip(true_matches["index_A"], true_matches["index_B"])
)

len(true_match_pairs)


100686

Here we see how many NPI's matching there are across data sets A and B. This will be used later checking how successful each blocking method is. 

In [8]:
blocking_cols = ["zip5"]

dataA_blk = dataA_prov.dropna(subset=blocking_cols).copy()
dataB_blk = dataB_prov.dropna(subset=blocking_cols).copy()

# Block sizes
block_sizes = (
    dataA_blk.groupby(blocking_cols)
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_blk.groupby(blocking_cols)
        .size()
        .reset_index(name="dataB_count"),
        on=blocking_cols,
        how="inner"
    )
)
block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

# Candidate pairs
exact_candidate_pairs = set()

groups_A = dataA_blk.groupby(blocking_cols)
groups_B = dataB_blk.groupby(blocking_cols)

common_keys = set(groups_A.groups) & set(groups_B.groups)

for key in common_keys:
    idx_A = groups_A.groups[key]
    idx_B = groups_B.groups[key]
    for i in idx_A:
        for j in idx_B:
            exact_candidate_pairs.add((i, j))

len(exact_candidate_pairs)


43176018

As we can see above there are over 43 million candidates. It is clear that this cannot be used but for the sake of completeness, I will check how successful this one is as well.

In [9]:
PC_exact_ZIP = (
    len(true_match_pairs & exact_candidate_pairs)
    / len(true_match_pairs)
)

PC_exact_ZIP

0.8059015156029636

In [10]:
# Blocking by ZIP code & initial of last name

for df in [dataA_prov, dataB_prov]:
    df["last_initial"] = df["Last.Name"].str[0]

block_sizes = (
    dataA_prov.groupby(["last_initial", "zip5"])
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_prov.groupby(["last_initial", "zip5"])
        .size()
        .reset_index(name="dataB_count"),
        on=["last_initial", "zip5"],
        how="inner"
    )
)

block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

block_sizes.sort_values("product", ascending=False).head(10)


/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/1996124010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["last_initial"] = df["Last.Name"].str[0]
/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_92735/1996124010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["last_initial"] = df["Last.Name"].str[0]


,last_initial,zip5,dataA_count,dataB_count,product
66137,S,77030,184,163,29992
47433,M,77030,162,136,22032
7971,B,77030,113,118,13334
12941,C,77030,106,110,11660
2974,A,77030,100,114,11400
42112,L,77030,101,86,8686
60676,R,77030,85,96,8160
64537,S,44195,118,69,8142
26206,G,77030,89,88,7832
56300,P,77030,86,88,7568


In [11]:
blocking_cols = ["last_initial","zip5"]

dataA_blk = dataA_prov.dropna(subset=blocking_cols).copy()
dataB_blk = dataB_prov.dropna(subset=blocking_cols).copy()

# Block sizes
block_sizes = (
    dataA_blk.groupby(blocking_cols)
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_blk.groupby(blocking_cols)
        .size()
        .reset_index(name="dataB_count"),
        on=blocking_cols,
        how="inner"
    )
)
block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

# Candidate pairs
exact_candidate_pairs = set()

groups_A = dataA_blk.groupby(blocking_cols)
groups_B = dataB_blk.groupby(blocking_cols)

common_keys = set(groups_A.groups) & set(groups_B.groups)

for key in common_keys:
    idx_A = groups_A.groups[key]
    idx_B = groups_B.groups[key]
    for i in idx_A:
        for j in idx_B:
            exact_candidate_pairs.add((i, j))

len(exact_candidate_pairs)


2604810

In [12]:
PC_exact_ZIP_Linitial = (
    len(true_match_pairs & exact_candidate_pairs)
    / len(true_match_pairs)
)

PC_exact_ZIP_Linitial

0.7961881492958306

In [38]:
# Blocking by ZIP code & initial of last name & initial of first name

for df in [dataA_prov, dataB_prov]:
    df["first_initial"] = df["First.Name"].str[0]

block_sizes = (
    dataA_prov.groupby(["first_initial","last_initial", "zip5"])
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_prov.groupby(["first_initial","last_initial", "zip5"])
        .size()
        .reset_index(name="dataB_count"),
        on=["first_initial","last_initial", "zip5"],
        how="inner"
    )
)

block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

block_sizes.sort_values("product", ascending=False).head(10)

# We reduce from trillions -> millions -> hundred thousands -> thousands 

/var/folders/c4/zrc90x8538v8_pcdjn16k8ww0000gn/T/ipykernel_44404/3404166453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["first_initial"] = df["First.Name"].str[0]


,first_initial,last_initial,zip5,dataA_count,dataB_count,product
98589,S,S,77030,27,21,567
9841,A,S,77030,28,18,504
89000,R,S,77030,22,20,440
51494,J,S,77030,17,21,357
74429,M,S,77030,22,15,330
95957,S,M,77030,20,14,280
71530,M,M,77030,19,14,266
48247,J,M,77030,16,15,240
90315,S,A,77030,16,14,224
96982,S,P,77030,15,14,210


In [39]:
blocking_cols = ["first_initial", "last_initial", "zip5"]

dataA_blk = dataA_prov.dropna(subset=blocking_cols).copy()
dataB_blk = dataB_prov.dropna(subset=blocking_cols).copy()

# Block sizes
block_sizes = (
    dataA_blk.groupby(blocking_cols)
    .size()
    .reset_index(name="dataA_count")
    .merge(
        dataB_blk.groupby(blocking_cols)
        .size()
        .reset_index(name="dataB_count"),
        on=blocking_cols,
        how="inner"
    )
)
block_sizes["product"] = (
    block_sizes["dataA_count"] * block_sizes["dataB_count"]
)

# Candidate pairs
exact_candidate_pairs = set()

groups_A = dataA_blk.groupby(blocking_cols)
groups_B = dataB_blk.groupby(blocking_cols)

common_keys = set(groups_A.groups) & set(groups_B.groups)

for key in common_keys:
    idx_A = groups_A.groups[key]
    idx_B = groups_B.groups[key]
    for i in idx_A:
        for j in idx_B:
            exact_candidate_pairs.add((i, j))

len(exact_candidate_pairs)


256669

In [40]:
PC_exact_ZIP_initials = (
    len(true_match_pairs & exact_candidate_pairs)
    / len(true_match_pairs)
)

PC_exact_ZIP_initials

0.7955525097828894

As we can see above that we reduced the number of candidate pairs from trillions to hundred thousands but while doing so we lose some precision. We lost almost $21\%$ of the true pairs. 

Now we will try another method. Specifically, we will try to implement sorted neighborhood method. Now on the sorted neighborhood method we will try using the all three columns we first intended to use. 

In [14]:
from itertools import combinations

def make_blocking_key(df):
    """
    Create a similarity-preserving blocking key
    """
    return (
        df["Last.Name"].str.upper().str.strip().str[:4].fillna("") +
        "_" +
        df["First.Name"].str.upper().str.strip().str[:1].fillna("") +
        "_" +
        df["State"].str.upper().fillna("")
    )


In [16]:
def SN_blocking(df_left, df_right, window_size=5):
    """
    Sorted neighborhood blocking across two datasets
    Returns list of (left_index, right_index) pairs
    """
    left = df_left.copy()
    right = df_right.copy()

    left["source"] = "L"
    right["source"] = "R"

    left["blocking_key"] = make_blocking_key(left)
    right["blocking_key"] = make_blocking_key(right)

    combined = pd.concat([left, right], axis=0)
    combined = combined.sort_values("blocking_key").reset_index(drop=False)

    candidate_pairs = []

    for i in range(len(combined)):
        window = combined.iloc[i : i + window_size]

        left_records = window[window["source"] == "L"]
        right_records = window[window["source"] == "R"]

        for l_idx in left_records["index"]:
            for r_idx in right_records["index"]:
                candidate_pairs.append((l_idx, r_idx))

    return candidate_pairs


In [17]:
SNB_candidate_pairs = set(
    SN_blocking(dataA_prov, dataB_prov, window_size=2)
)

len(SNB_candidate_pairs)


383219

In [18]:
PC_SNB = (
    len(true_match_pairs & SNB_candidate_pairs)
    / len(true_match_pairs)
)

PC_SNB


0.7750432036231453

In [19]:
SNB_candidate_pairs = set(
    SN_blocking(dataA_prov, dataB_prov, window_size=5)
)

len(SNB_candidate_pairs)


1434608

In [20]:
PC_SNB = (
    len(true_match_pairs & SNB_candidate_pairs)
    / len(true_match_pairs)
)

PC_SNB


0.9349462685974217

As we can see that SNB method did work using the State column and it gave us 1.4 million candidate pairs, which is not bad at all. Now we have to see how precise this method compared to exact blocking method.

It is rather obvious that SNB is supreior over the exact blocking. While getting the pairs using this method, we only lost $7\%$ of the actual pairs. That's why we decided to use this method later in the analysis.

In [41]:
SNB_candidate_pairs

{(389342, 26016),
 (323512, 109796),
 (150969, 121193),
 (381482, 645658),
 (78114, 610059),
 (676012, 4807),
 (515831, 52638),
 (235685, 295921),
 (240575, 652994),
 (107137, 59469),
 (30881, 340374),
 (75786, 619949),
 (17553, 183650),
 (490569, 115500),
 (2097, 52712),
 (587778, 19867),
 (304149, 326036),
 (566592, 114318),
 (107617, 31998),
 (282017, 120124),
 (327384, 66091),
 (156316, 299318),
 (6525, 486507),
 (15359, 427236),
 (134835, 165605),
 (39923, 467392),
 (48993, 250546),
 (166779, 29644),
 (315608, 45215),
 (228271, 290574),
 (182456, 190537),
 (57896, 446453),
 (502680, 474061),
 (612690, 193890),
 (156423, 423674),
 (369779, 304022),
 (592176, 122194),
 (268221, 482025),
 (306774, 111252),
 (591594, 76203),
 (191034, 593400),
 (308829, 163993),
 (315640, 96027),
 (99895, 107223),
 (22123, 257572),
 (151087, 94780),
 (262365, 8045),
 (398403, 182077),
 (399043, 625690),
 (532916, 173789),
 (581905, 381084),
 (33576, 601261),
 (14882, 364800),
 (697653, 264738),
 (4880

In [192]:
common_element = next(iter(set(dataA['NPI']) & set(dataB['NPI'])), None)

print(common_element)

idxA = dataA.index[dataA['NPI'] == common_element][0]
idxB = dataB.index[dataB['NPI'] == common_element][0]

print(idxA) 
print(idxB) 

print(dataA.iloc[103774,:])

print(dataB.iloc[19249,:])




1184628737.0
103774
19249
NPI                         1184628737
Last.Name                        BROWN
First.Name                       BARRY
Middle.Name                          S
Credentials                         DO
City                          LOW MOOR
State                               VA
ZIP                                nan
Country                             US
Type                 INTERNAL MEDICINE
Row.Index                      1775458
Street.Address    200 ARH LANE STE 100
Name: 103774, dtype: object
NPI                                                    1184628737.0
First.Name                                                    BARRY
Middle.Name                                                     NaN
Last.Name                                                     BROWN
City                                                       LOW MOOR
State                                                            VA
ZIP                                                           24457
Co

## Intermediate Cleaning

Now that we are done with blocking part, we will form the similarity tables. However, before that we will actually a little bit more data cleaning here as well. 

In [21]:
# Cleaning data set B

dataB=dataB.drop(columns=["Type","Profile.ID","Name.Suffix","Payment.ID","Payment","Payment.Date","Record.ID",
                          "Program.Year","Payment.Publication.Date","Postal.Code","Business.Name","Province"])

dataC=dataC.drop(columns=["Multiple.NPI","PECOS.ID","Enrollment.ID","Type.Code","Organization.Name"])

dataB_test=dataB_test.drop(columns=["Type","Profile.ID","Name.Suffix","Payment.ID","Payment","Payment.Date","Record.ID",
                          "Program.Year","Payment.Publication.Date","Postal.Code","Business.Name","Province"])

dataC_test=dataC_test.drop(columns=["Multiple.NPI","PECOS.ID","Enrollment.ID","Type.Code","Organization.Name"])

dataB = dataB.rename(columns={"Primary.Type":"Credentials"})
dataB = dataB.rename(columns={"Specialty":"Type"})

dataB_test = dataB_test.rename(columns={"Primary.Type":"Credentials"})
dataB_test = dataB_test.rename(columns={"Specialty":"Type"})

dataB['Credentials'] = dataB['Credentials'].str.findall(r'\b\w').str.join('').str.upper()
dataB['Country'] = dataB['Country'].str.findall(r'\b\w').str.join('').str.upper()

dataB_test['Credentials'] = dataB_test['Credentials'].str.findall(r'\b\w').str.join('').str.upper()
dataB_test['Country'] = dataB_test['Country'].str.findall(r'\b\w').str.join('').str.upper()

dataB = dataB.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataB_test = dataB_test.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataB["ZIP"] = dataB["ZIP"].map(str)

dataB_test["ZIP"] = dataB_test["ZIP"].map(str)

dataB['Street.Address'] = dataB['Street.Address'].str.replace(r'[^\w\s]', '', regex=True)

dataB_test['Street.Address'] = dataB_test['Street.Address'].str.replace(r'[^\w\s]', '', regex=True)

dataB['Type'] = dataB['Type'].str.replace(r'[^\w\s]', ' ', regex=True)

dataB_test['Type'] = dataB_test['Type'].str.replace(r'[^\w\s]', ' ', regex=True)


In [22]:
# Cleaning data set A

dataA = dataA.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataA_test = dataA_test.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataA['Credentials'] = dataA['Credentials'].str.replace(r'[^\w\s]', '', regex=True)

dataA_test['Credentials'] = dataA_test['Credentials'].str.replace(r'[^\w\s]', '', regex=True)

dataA["ZIP"] = dataA["ZIP"].map(str)

dataA_test["ZIP"] = dataA_test["ZIP"].map(str)

dataA['Street.Address'] = dataA['Street.Address'].str.replace(r'[^\w\s]', '', regex=True)

dataA_test['Street.Address'] = dataA_test['Street.Address'].str.replace(r'[^\w\s]', '', regex=True)

dataA['Type'] = dataA['Type'].str.replace(r'[^\w\s]', ' ', regex=True)

dataA_test['Type'] = dataA_test['Type'].str.replace(r'[^\w\s]', ' ', regex=True)

In [23]:
# Cleaning data set C

dataC = dataC.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataC_test = dataC_test.apply(lambda col: col.str.upper() if col.dtype == 'object' else col)

dataC["ZIP"] = dataC["ZIP"].map(str)

dataC_test["ZIP"] = dataC_test["ZIP"].map(str)

dataC['Type'] = dataC['Type'].str.replace(r'[^\w\s]', ' ', regex=True)

dataC_test['Type'] = dataC_test['Type'].str.replace(r'[^\w\s]', ' ', regex=True)

## Similarity Table

Now we will create the similarity tables to use in machine learning. Here for different columns we will use different type of similarity functions/methods. 

In [125]:
def NPI_check(i,j):
    if dataA["NPI"][i]==dataB["NPI"][j]:
        return 1
    else:
        return 0
    

In [216]:
import textdistance 

JW = textdistance.JaroWinkler()
lev = textdistance.Levenshtein()

def safe_jw(a, b):
    if not isinstance(a, str) or not isinstance(b, str):
        return 0.0
    return JW(a, b)


def safe_lev(a, b):
    if not isinstance(a, str) or not isinstance(b, str):
        return 0.0
    return lev.normalized_similarity(a, b)


Below what I'm doing is trying to find the most common words that appear on Street.Address column, so that I can drop them. The reason is that since they appear very commonly they will spoil the similarity analysis. 

In [162]:
from collections import Counter
import re

tokens = (
    dataA["Street.Address"]
    .str.findall(r"\b\w+\b")   
    .explode()
)

token_counts = Counter(tokens)

token_counts.most_common(20)

[('ST', 158906),
 ('STE', 157653),
 ('AVE', 125621),
 ('RD', 115973),
 ('SUITE', 93699),
 ('DR', 86868),
 ('BLVD', 59994),
 ('N', 59233),
 ('W', 56006),
 ('E', 54867),
 ('S', 52403),
 ('200', 27495),
 ('100', 27462),
 ('MEDICAL', 19770),
 ('PKWY', 17001),
 ('CENTER', 16870),
 ('1', 16276),
 ('300', 15354),
 ('PARK', 14014),
 ('101', 13774)]

In [163]:
tokens = (
    dataB["Street.Address"]
    .str.findall(r"\b\w+\b")   
    .explode()
)

token_counts = Counter(tokens)

token_counts.most_common(20)

[('STE', 255484),
 ('ST', 143939),
 ('RD', 125187),
 ('AVE', 122292),
 ('DR', 92919),
 ('BLVD', 62382),
 ('SUITE', 60443),
 ('N', 59985),
 ('W', 53699),
 ('E', 52824),
 ('S', 50123),
 ('100', 29752),
 ('200', 28922),
 ('PKWY', 21042),
 ('300', 16024),
 ('MEDICAL', 15446),
 ('101', 15129),
 ('PARK', 14499),
 ('1', 14027),
 ('A', 12552)]

We see that top 11 words in Street.Address column on both data sets A and B are the same. That's why we will drop them.

Below is the function I wrote for street address similarity. It first tokenizes everything from the Street.Address columns and it also calculates the stats of tokens appearing. Then for all the tokens that are rare, we check their similarities using both Jaro-Winkler and Levenshtein. If the words are similar to each other, then we record those words as the same, this is the step of creating canonical map. Then we compare two addresses by using TF-IDF vectorization in cosine similarity. 

In [201]:
words_to_drop = [pair[0] for pair in token_counts.most_common(11)]
words_to_drop.extend(["STREET", "BOULEVARD","ROAD","AVENUE","NORTH","WEST","EAST","SOUTH","DRIVE"])

JW_THRESHOLD = 0.85
LEV_THRESHOLD = 0.20
MAX_RARE_FREQ = 1 

def tokenize_address(addr):
    if not isinstance(addr, str):
        return []

    tokens = re.findall(r"\b\w+\b", addr)
    return [t for t in tokens if t not in words_to_drop]

def build_token_stats(addresses):
    token_lists = [tokenize_address(a) for a in addresses]
    flat_tokens = [t for tokens in token_lists for t in tokens]
    freq = Counter(flat_tokens)
    return token_lists, freq

def get_rare_tokens(freq):
    return [t for t, c in freq.items() if c <= MAX_RARE_FREQ]

def similar_tokens(s1, s2):
    if s1.isdigit() or s2.isdigit():
        return False

    jw = JW(s1, s2)
    lev_dist = lev.normalized_distance(s1,s2)

    return jw >= JW_THRESHOLD or lev_dist <= LEV_THRESHOLD

def build_canonical_map(rare_tokens):
    canonical = {}
    used = set()

    for t1, t2 in combinations(rare_tokens, 2):
        if t1 in used or t2 in used:
            continue

        if similar_tokens(t1, t2):
            # choose longer token as canonical (heuristic)
            canon = t1 if len(t1) >= len(t2) else t2
            other = t2 if canon == t1 else t1

            canonical[other] = canon
            used.add(other)

    return canonical

def canonicalize_tokens(token_lists, canon_map):
    new_lists = []
    for tokens in token_lists:
        new_lists.append([canon_map.get(t, t) for t in tokens])
    return new_lists

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def compare_two_addresses(addr1, addr2, canon_map):
    tokens1 = canonicalize_tokens([tokenize_address(addr1)], canon_map)[0]
    tokens2 = canonicalize_tokens([tokenize_address(addr2)], canon_map)[0]

    docs = [" ".join(tokens1), " ".join(tokens2)]

    vectorizer = TfidfVectorizer(
        tokenizer=str.split,
        preprocessor=None,
        lowercase=False
    )

    X = vectorizer.fit_transform(docs)
    return cosine_similarity(X)[0, 1]





In [200]:
all_addresses = list(dataA['Street.Address']) + list(dataB['Street.Address'])

token_lists, freq = build_token_stats(all_addresses)
rare_tokens = get_rare_tokens(freq)
canonical_map = build_canonical_map(rare_tokens)



Next function is for checking the similarity between Type columns. This function primarly checks if two strings contains on word which is "similar". And bascially it returns 1 if yes an returns 0 if no.

In [210]:
def tokenize(s):
    if not isinstance(s, str):
        return []
    return re.findall(r"\b\w+\b", s)

def matching_words(s1, s2, prefix_len=3, jw_threshold=0.85):
    tokens1 = tokenize(s1)
    tokens2 = tokenize(s2)

    # Build prefix → words mapping for s2
    prefix_map = {}
    for w in tokens2:
        if len(w) >= prefix_len:
            prefix = w[:prefix_len]
            prefix_map.setdefault(prefix, []).append(w)

    # Compare tokens from s1
    for w1 in tokens1:
        if len(w1) < prefix_len:
            continue

        prefix = w1[:prefix_len]
        candidates = prefix_map.get(prefix, [])

        for w2 in candidates:
            score = JW(w1, w2)
            if score >= jw_threshold:
                return 1

    return 0


In [217]:
rows = []

for pair in SNB_candidate_pairs:
    row = []

    row.append(pair)
    row.append(NPI_check(pair[0],pair[1]))
    row.append(safe_jw(dataA["First.Name"][pair[0]],dataB["First.Name"][pair[1]]))
    row.append(safe_jw(dataA["Last.Name"][pair[0]],dataB["Last.Name"][pair[1]]))
    row.append(safe_lev(dataA["Middle.Name"][pair[0]],dataB["Middle.Name"][pair[1]]))
    row.append(safe_lev(dataA["Credentials"][pair[0]],dataB["Credentials"][pair[1]]))
    row.append(safe_jw(dataA["City"][pair[0]],dataB["City"][pair[1]]))
    row.append(safe_lev(dataA["State"][pair[0]],dataB["State"][pair[1]]))
    row.append(safe_lev(dataA["Country"][pair[0]],dataB["Country"][pair[1]]))
    row.append(safe_jw(dataA["ZIP"][pair[0]],dataB["ZIP"][pair[1]]))
    row.append(compare_two_addresses(dataA['Street.Address'][pair[0]],dataB['Street.Address'][pair[1]],canonical_map))
    row.append(matching_words(dataA['Type'][pair[0]],dataB['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

df = pd.DataFrame(rows, columns=columns)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [218]:
display(df)


,pair,NPI_match,First.Name,Last.Name,Middle.Name,Credentials,City,State,Country,ZIP,Street.Address,Type
0,"(389342, 26016)",0,0.950000,0.847619,0.0,0.000000,0.547619,0.0,1.0,0.600000,0.000000,1
1,"(323512, 109796)",1,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,0.900000,0.776515,1
2,"(150969, 121193)",0,1.000000,1.000000,0.0,0.333333,0.398810,0.5,1.0,0.466667,0.000000,0
3,"(381482, 645658)",0,0.472222,0.634921,0.0,1.000000,0.544444,0.0,1.0,0.600000,0.000000,0
4,"(78114, 610059)",0,0.595238,0.699634,0.0,0.000000,0.522222,0.0,1.0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1434603,"(29114, 176638)",0,0.537879,0.836111,0.0,1.000000,0.000000,0.0,1.0,0.000000,0.000000,0
1434604,"(607598, 654946)",0,0.472222,0.880455,0.0,1.000000,0.419192,0.0,1.0,0.622222,0.000000,0
1434605,"(221280, 112068)",0,0.000000,0.790000,0.0,0.333333,0.555556,0.0,1.0,0.000000,0.000000,0
1434606,"(464048, 224212)",0,0.539683,0.914286,0.0,0.000000,0.430556,0.0,1.0,0.000000,0.000000,0


In [221]:
len(df[df['NPI_match']==1])

94136

## Model Selection

Before we create the other similarity tables (for A vs C and B vs C), we will first select our model by using cross validation. Later, we will train the model using all three similarity tables. While using cross validation we are actually using stratified k-folds and the reason is that we saw above that there are 94k NPI matches in 1.4m rows. Hence, those 94k matches should be spread somewhat equally to each fold. 

We will first start by logistic regression. 

In [222]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score

X = df.drop(columns=["pair", "NPI_match"])
y = df["NPI_match"]

model = LogisticRegression(
    class_weight="balanced",
    max_iter=1000,
    n_jobs=-1
)

cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scoring = {
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "f1": make_scorer(f1_score),
    "roc_auc": "roc_auc"
}

results = cross_validate(
    model,
    X,
    y,
    cv=cv,
    scoring=scoring,
    return_train_score=False
)

In [224]:
for metric in ["test_precision", "test_recall", "test_f1", "test_roc_auc"]:
    mean = np.mean(results[metric])
    std = np.std(results[metric])
    print(f"{metric}: {mean:.4f} ± {std:.4f}")

test_precision: 0.8974 ± 0.0020
test_recall: 0.9945 ± 0.0005
test_f1: 0.9434 ± 0.0012
test_roc_auc: 0.9997 ± 0.0000


Now we will check random forest classifier.

In [288]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=50,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics_rf = {
    "precision": [],
    "recall": [],
    "f1": [],
    "roc_auc": []
}

for train_idx, val_idx in skf.split(X, y):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    rf.fit(X_tr, y_tr)

    y_pred = rf.predict(X_val)
    y_prob = rf.predict_proba(X_val)[:, 1]

    metrics_rf["precision"].append(precision_score(y_val, y_pred))
    metrics_rf["recall"].append(recall_score(y_val, y_pred))
    metrics_rf["f1"].append(f1_score(y_val, y_pred))
    metrics_rf["roc_auc"].append(roc_auc_score(y_val, y_prob))




In [228]:
for m, vals in metrics.items():
    print(f"{m}: {np.mean(vals):.4f} ± {np.std(vals):.4f}")


precision: 0.9275 ± 0.0007
recall: 0.9945 ± 0.0003
f1: 0.9598 ± 0.0005
roc_auc: 0.9998 ± 0.0000


Finally, we will check the gradient boosting classifier. 

In [229]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

gb = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics = {
    "precision": [],
    "recall": [],
    "f1": [],
    "roc_auc": []
}

for train_idx, val_idx in skf.split(X, y):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    gb.fit(X_tr, y_tr)

    y_pred = gb.predict(X_val)
    y_prob = gb.predict_proba(X_val)[:, 1]

    metrics["precision"].append(precision_score(y_val, y_pred))
    metrics["recall"].append(recall_score(y_val, y_pred))
    metrics["f1"].append(f1_score(y_val, y_pred))
    metrics["roc_auc"].append(roc_auc_score(y_val, y_prob))




In [230]:
for m, vals in metrics.items():
    print(f"{m}: {np.mean(vals):.4f} ± {np.std(vals):.4f}")


precision: 0.9844 ± 0.0006
recall: 0.9718 ± 0.0013
f1: 0.9781 ± 0.0008
roc_auc: 0.9998 ± 0.0000


Now we will check if gradient boosting is significantly better than the other two models. We will check this using statistical hypothesis testing. Here our null hypothesis is gradient boosting is not much different than the other methods. 

In [302]:
lr_precision = results["test_precision"]
lr_recall = results["test_recall"]
lr_f1 = results['test_f1']
lr_roc_auc = results['test_roc_auc']

lr_f1

array([0.94268158, 0.94451587, 0.94238414, 0.94223118, 0.94528845])

In [296]:
rf_precision = metrics_rf["precision"]
rf_recall = metrics_rf["recall"]
rf_f1 = metrics_rf["f1"]
rf_roc_auc = metrics_rf['roc_auc']


[0.959729307118505,
 0.9601518026565465,
 0.959870848708487,
 0.9589525468894128,
 0.9603671041837571]

In [297]:
gb_precision = metrics["precision"]
gb_recall = metrics["recall"]
gb_f1 = metrics["f1"]
gb_roc_auc = metrics['roc_auc']

[0.9767342354388404,
 0.9787484295223075,
 0.9789085860743272,
 0.9775425088225859,
 0.9783718754177249]

In [303]:
from scipy.stats import wilcoxon

stat_gb_lr, pval_gb_lr = wilcoxon(
    gb_f1,
    lr_f1,
    alternative="greater"
)

print(f"GB vs LR Wilcoxon p-value: {pval_gb_lr}")


GB vs LR Wilcoxon p-value: 0.03125


In [304]:
stat_gb_rf, pval_gb_rf = wilcoxon(
    gb_f1,
    rf_f1,
    alternative="greater"
)

print(f"GB vs RF Wilcoxon p-value: {pval_gb_rf}")


GB vs RF Wilcoxon p-value: 0.03125


By looking at the results, since they are smaller than 0.05, we can statistically say that gradient boosting is significantly better than other two classifiers. That's why we will choose that one while training our model. Now within this model we will check the hyper parameters by using GridSearchCV.

In [346]:
param_grid = {
    "n_estimators": [200, 300],
    "learning_rate": [0.03, 0.05],
    "max_depth": [2, 3]
}


In [ ]:
from sklearn.model_selection import GridSearchCV

gb = GradientBoostingClassifier(
    random_state=42
)

cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

grid = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    scoring="f1",            
    cv=cv,
    n_jobs=-1,
    verbose=2
)


In [348]:
X = df.drop(columns=["pair", "NPI_match"])
y = df["NPI_match"]

grid.fit(X, y)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=200; total time= 4.1min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=200; total time= 4.1min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=200; total time= 4.1min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=200; total time= 4.1min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=200; total time= 4.2min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=300; total time= 6.3min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=300; total time= 6.3min
[CV] END ..learning_rate=0.03, max_depth=2, n_estimators=300; total time= 6.3min
[CV] END ..learning_rate=0.03, max_depth=3, n_estimators=200; total time= 5.9min
[CV] END ..learning_rate=0.03, max_depth=3, n_estimators=200; total time= 5.9min
[CV] END ..learning_rate=0.03, max_depth=3, n_estimators=200; total time= 6.0min
[CV] END ..learning_rate=0.03, max_depth=2, n_est

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.03, 0.05], 'max_depth': [2, 3],
                         'n_estimators': [200, 300]},
             scoring='f1', verbose=2)

In [349]:
grid.best_params_


{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300}

In [350]:
grid.best_score_


0.9780939715150965

In [351]:
best_model = grid.best_estimator_


Now we know which hyper parameters to use while training our model on the whole similarity tables.

## Training the Model

We will first create the same similarity table for the data sets A and C

In [309]:
def NPI_check_AC(i,j):
    if dataA["NPI"][i]==dataC["NPI"][j]:
        return 1
    else:
        return 0

In [305]:
SNB_candidate_pairs_AC = set(
    SN_blocking(dataA_prov, dataC_prov, window_size=5)
)

len(SNB_candidate_pairs_AC)

1824236

In [313]:
rows = []

for pair in SNB_candidate_pairs_AC:
    row = []

    row.append(pair)
    row.append(NPI_check_AC(pair[0],pair[1]))
    row.append(safe_jw(dataA["First.Name"][pair[0]],dataC["First.Name"][pair[1]]))
    row.append(safe_jw(dataA["Last.Name"][pair[0]],dataC["Last.Name"][pair[1]]))
    row.append(safe_lev(dataA["Middle.Name"][pair[0]],dataC["Middle.Name"][pair[1]]))
    row.append(0) # no credentials in dataC
    row.append(safe_jw(dataA["City"][pair[0]],dataC["City"][pair[1]]))
    row.append(safe_lev(dataA["State"][pair[0]],dataC["State"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(safe_jw(dataA["ZIP"][pair[0]],dataC["ZIP"][pair[1]]))
    row.append(0) # no address in dataC 
    row.append(matching_words(dataA['Type'][pair[0]],dataC['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

df_AC = pd.DataFrame(rows, columns=columns)


Let's do the same thing for data sets B and C.

In [319]:
def NPI_check_BC(i,j):
    if dataB["NPI"][i]==dataC["NPI"][j]:
        return 1
    else:
        return 0

In [316]:
SNB_candidate_pairs_BC = set(
    SN_blocking(dataB_prov, dataC_prov, window_size=5)
)

len(SNB_candidate_pairs_BC)

1498706

In [320]:
rows = []

for pair in SNB_candidate_pairs_BC:
    row = []

    row.append(pair)
    row.append(NPI_check_BC(pair[0],pair[1]))
    row.append(safe_jw(dataB["First.Name"][pair[0]],dataC["First.Name"][pair[1]]))
    row.append(safe_jw(dataB["Last.Name"][pair[0]],dataC["Last.Name"][pair[1]]))
    row.append(safe_lev(dataB["Middle.Name"][pair[0]],dataC["Middle.Name"][pair[1]]))
    row.append(0) # no credentials in dataC
    row.append(safe_jw(dataB["City"][pair[0]],dataC["City"][pair[1]]))
    row.append(safe_lev(dataB["State"][pair[0]],dataC["State"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(safe_jw(dataB["ZIP"][pair[0]],dataC["ZIP"][pair[1]]))
    row.append(0) # no address in dataC 
    row.append(matching_words(dataB['Type'][pair[0]],dataC['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

df_BC = pd.DataFrame(rows, columns=columns)


Now we will put these three similarity table on top of each other. 

In [324]:
df_new = pd.concat([df,df_AC,df_BC],axis=0)

And now we will train our model on this new data using the hyper parameters we found. 

In [326]:
X_all = df_new.drop(columns=["pair", "NPI_match"])
y_all = df_new["NPI_match"]

final_model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

final_model.fit(X_all, y_all)


GradientBoostingClassifier(learning_rate=0.05, n_estimators=300,
                           random_state=42, subsample=0.8)

We will now try to interpret our model by checking the feature importances.

In [353]:
feature_importance = pd.DataFrame({
    "feature": X_all.columns,
    "importance": final_model.feature_importances_
}).sort_values(by="importance", ascending=False)

feature_importance


,feature,importance
0,First.Name,0.657323
5,State,0.193077
9,Type,0.060079
1,Last.Name,0.052447
4,City,0.014711
8,Street.Address,0.013428
2,Middle.Name,0.008327
6,Country,0.000327
7,ZIP,0.000150
3,Credentials,0.000131


## Testing the Model

Now we will test our model on the test data. However, we again need to first create the three similarity tables. Then we will stack them on top of each other. 

Let's first create the similarity table for test data set A and B. 

In [272]:
def NPI_check_test(i,j):
    if dataA_test["NPI"][i]==dataB_test["NPI"][j]:
        return 1
    else:
        return 0

In [24]:
SNB_test_pairs = set(
    SN_blocking(dataA_test, dataB_test, window_size=5)
)

len(SNB_test_pairs)

1068108

In [273]:
rows = []

for pair in SNB_test_pairs:
    row = []

    row.append(pair)
    row.append(NPI_check_test(pair[0],pair[1]))
    row.append(safe_jw(dataA_test["First.Name"][pair[0]],dataB_test["First.Name"][pair[1]]))
    row.append(safe_jw(dataA_test["Last.Name"][pair[0]],dataB_test["Last.Name"][pair[1]]))
    row.append(safe_lev(dataA_test["Middle.Name"][pair[0]],dataB_test["Middle.Name"][pair[1]]))
    row.append(safe_lev(dataA_test["Credentials"][pair[0]],dataB_test["Credentials"][pair[1]]))
    row.append(safe_jw(dataA_test["City"][pair[0]],dataB_test["City"][pair[1]]))
    row.append(safe_lev(dataA_test["State"][pair[0]],dataB_test["State"][pair[1]]))
    row.append(safe_lev(dataA_test["Country"][pair[0]],dataB_test["Country"][pair[1]]))
    row.append(safe_jw(dataA_test["ZIP"][pair[0]],dataB_test["ZIP"][pair[1]]))
    row.append(compare_two_addresses(dataA_test['Street.Address'][pair[0]],dataB_test['Street.Address'][pair[1]],canonical_map))
    row.append(matching_words(dataA_test['Type'][pair[0]],dataB_test['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

test_data = pd.DataFrame(rows, columns=columns)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Let's do the same thing for test data A and C.

In [329]:
def NPI_check_test_AC(i,j):
    if dataA_test["NPI"][i]==dataC_test["NPI"][j]:
        return 1
    else:
        return 0

In [328]:
SNB_test_pairs_AC = set(
    SN_blocking(dataA_test, dataC_test, window_size=5)
)

len(SNB_test_pairs_AC)

962514

In [333]:
rows = []

for pair in SNB_test_pairs_AC:
    row = []

    row.append(pair)
    row.append(NPI_check_test_AC(pair[0],pair[1]))
    row.append(safe_jw(dataA_test["First.Name"][pair[0]],dataC_test["First.Name"][pair[1]]))
    row.append(safe_jw(dataA_test["Last.Name"][pair[0]],dataC_test["Last.Name"][pair[1]]))
    row.append(safe_lev(dataA_test["Middle.Name"][pair[0]],dataC_test["Middle.Name"][pair[1]]))
    row.append(0) # no credentials in dataC
    row.append(safe_jw(dataA_test["City"][pair[0]],dataC_test["City"][pair[1]]))
    row.append(safe_lev(dataA_test["State"][pair[0]],dataC_test["State"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(safe_jw(dataA_test["ZIP"][pair[0]],dataC_test["ZIP"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(matching_words(dataA_test['Type'][pair[0]],dataC_test['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

test_data_AC = pd.DataFrame(rows, columns=columns)


Finally, let's do the same thing for test data B and C.

In [337]:
def NPI_check_test_BC(i,j):
    if dataB_test["NPI"][i]==dataC_test["NPI"][j]:
        return 1
    else:
        return 0

In [336]:
SNB_test_pairs_BC = set(
    SN_blocking(dataB_test, dataC_test, window_size=5)
)

len(SNB_test_pairs_BC)

940604

In [338]:
rows = []

for pair in SNB_test_pairs_BC:
    row = []

    row.append(pair)
    row.append(NPI_check_test_BC(pair[0],pair[1]))
    row.append(safe_jw(dataB_test["First.Name"][pair[0]],dataC_test["First.Name"][pair[1]]))
    row.append(safe_jw(dataB_test["Last.Name"][pair[0]],dataC_test["Last.Name"][pair[1]]))
    row.append(safe_lev(dataB_test["Middle.Name"][pair[0]],dataC_test["Middle.Name"][pair[1]]))
    row.append(0) # no credentials in dataC
    row.append(safe_jw(dataB_test["City"][pair[0]],dataC_test["City"][pair[1]]))
    row.append(safe_lev(dataB_test["State"][pair[0]],dataC_test["State"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(safe_jw(dataB_test["ZIP"][pair[0]],dataC_test["ZIP"][pair[1]]))
    row.append(0) # no country in dataC
    row.append(matching_words(dataB_test['Type'][pair[0]],dataC_test['Type'][pair[1]]))

    rows.append(row)
    
    
columns = [
    "pair",
    "NPI_match",
    "First.Name",
    "Last.Name",
    "Middle.Name",
    "Credentials",
    "City",
    "State",
    "Country",
    "ZIP",
    "Street.Address",
    "Type"
]

test_data_BC = pd.DataFrame(rows, columns=columns)


Now we will stack them on top of each other. 

In [354]:
test = pd.concat([test_data,test_data_AC,test_data_BC],axis=0)

In [355]:
X_test = test.drop(columns=["pair", "NPI_match"])
y_test = test["NPI_match"]

In [356]:
y_test_pred = final_model.predict(X_test)
y_test_prob = final_model.predict_proba(X_test)[:, 1]

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_prob)

print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1:        {test_f1:.4f}")
print(f"Test ROC AUC:   {test_auc:.4f}")

Test Precision: 0.9635
Test Recall:    0.9466
Test F1:        0.9550
Test ROC AUC:   0.9996


We will apply bootstrping here to check how much my test results changed if my test data changed slightly. 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Predictions
y_prob = final_model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

# Convert to NumPy 
y_true = y_test.to_numpy()
y_pred = y_pred.astype(np.int8)
y_prob = y_prob.astype(np.float32)

n = y_true.shape[0]
n_bootstraps = 500  
rng = np.random.default_rng(42)

# Preallocate arrays
precisions = np.empty(n_bootstraps)
recalls = np.empty(n_bootstraps)
f1s = np.empty(n_bootstraps)
aucs = np.empty(n_bootstraps)

for i in range(n_bootstraps):
    idx = rng.integers(0, n, size=n)

    y_true_bs = y_true[idx]
    y_pred_bs = y_pred[idx]
    y_prob_bs = y_prob[idx]

    precisions[i] = precision_score(y_true_bs, y_pred_bs, zero_division=0)
    recalls[i] = recall_score(y_true_bs, y_pred_bs)
    f1s[i] = f1_score(y_true_bs, y_pred_bs)
    aucs[i] = roc_auc_score(y_true_bs, y_prob_bs)

# Confidence interval function
def ci(x):
    return np.percentile(x, [2.5, 97.5])

print("Precision CI:", ci(precisions))
print("Recall CI:   ", ci(recalls))
print("F1 CI:       ", ci(f1s))
print("ROC AUC CI:  ", ci(aucs))


Precision CI: [0.96249332 0.96449145]
Recall CI:    [0.94539686 0.94784002]
F1 CI:        [0.9542114  0.95586109]
ROC AUC CI:   [0.99958322 0.99961113]


Realize that all of the original results lie in between the given confidence intervals. That means our model did a great job learning the behaviour of the training data.